In [4]:
import numpy as np
import plotly.graph_objects as go
# import soundfile as sf
import wave
import sys
from pathlib import Path
import sound_card_tools as sct
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import spectrogram

show_time_plot = False

# Open the wav file
filename = 'sound_test_with_sine.wav'

# sound_test_with_sine_20250108.wav
# delay_cut_idx = delay_idx  - 12 
delay_cut_idx_offset = 12 
# now further cut the cut signals by only keeping the samples [60000:120000]
# sc_cut_input = sc_cut_input[60000:120000]
# sc_cut_input_range 60000:120000
# sc_cut_output = sc_cut_output[60000:120000]
sc_cut_range = range(60000,120000)

#sc_cut_input, sc_cut_output = sct.cut_signals(sc_input, sc_output, sample_rate*15)
sc_cut_in_sec = 15

# shift = sct.find_shift_between_signals(sc_cut_output[0:800], 
#                              sc_cut_input[0:800],  
#                              sample_rate=sample_rate,
shift_between_signals_range = range(0,800)
interpolated_shift_factor = 10
freq_diff_bounds_hz=[3800, 4400]
#phase_diff_bounds_rad=[-7, -5]

filename = 'sound_test_with_sine_baseline_20250112.wav'
phase_diff_bounds_rad=None
sc_cut_in_sec = None
shift_between_signals_range = range(600_000,610_000)
plot_range = range(400_000, 800_000)

if False:
    filename = 'sound_test_with_sine_ice_20250112.wav'
    shift_between_signals_range = range(1_000_000,1_010_000)
    plot_range = range(1_000_000, 1_700_000)

file_path = Path('/home/user/data')
full_file_path = file_path / filename
                    

# Open the wav file
wav_file = wave.open(str(full_file_path), 'r')

# Extract Raw Audio from Wav File
signal = wav_file.readframes(-1)
signal = np.frombuffer(signal, dtype=np.int16)

# Get the number of channels
channels = wav_file.getnchannels()

# Split the data into channels
if channels == 2:
    signal = np.reshape(signal, (-1, 2))
    sc_output = signal[:, 0]
    sc_input = signal[:, 1]
else:
    print('The file does not have 2 channels')
    sys.exit(0)
# get the sample rate of the wav file
sample_rate = wav_file.getframerate()
 


# Create interactive plot
if show_time_plot:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=sc_output, mode='lines', name='Sound Card Output'))
    fig.add_trace(go.Scatter(y=sc_input, mode='lines', name='Sound Card Input'))
    fig.update_layout(title='Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
    fig.show()



In [ ]:
# cut both sc_input and sc_output to the same length with sct.cut_signals
[delay_idx, delay_seconds]= sct.find_delay_by_amplitude(sc_output, sc_input, sample_rate,noise_window=1000, threshold_factor=15)
delay_cut_idx = delay_idx  - delay_cut_idx_offset
sc_cut_input, sc_cut_output = sct.cut_signals(sc_input, sc_output, delay_cut_idx)

# now further cut the cut signals by only keeping the samples [60000:120000]
sc_cut_input = sc_cut_input[sc_cut_range]
sc_cut_output = sc_cut_output[sc_cut_range]

# plot the cut signals
fig = go.Figure()
fig.add_trace(go.Scatter(y=sc_cut_output, mode='lines', name='sc_output'))
fig.add_trace(go.Scatter(y=sc_cut_input, mode='lines', name='sc_input'))
fig.update_layout(title='Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
fig.show()



In [3]:


# # Generate spectrogram for sc_output
frequencies_output, times_output, Sxx_output = spectrogram(sc_output, fs=sample_rate)

# # Generate spectrogram for sc_input
frequencies_input, times_input, Sxx_input = spectrogram(sc_input, fs=sample_rate)

In [ ]:

spec_out_db = 10 * np.log10(Sxx_output)
spec_in_db = 10 * np.log10(Sxx_input)

# Find global min and max across both spectrograms
global_min = min(spec_out_db.min(), spec_in_db.min())
global_max = max(spec_out_db.max(), spec_in_db.max())

z_scale_min_max = [global_max-70, global_max]


# -----------------------------------------------------
# PLOT: Time-domain plot + Spectrogram for sc_input
# with linked (shared) x-axis
# -----------------------------------------------------

# 1) Create subplots with shared x-axis
fig_input = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,         # crucial for linking
    vertical_spacing=0.12      # adjust as needed
)

# 2) Top subplot: time-domain plot of sc_input
time_array = np.arange(len(sc_input)) / sample_rate
fig_input.add_trace(
    go.Scatter(
        x=time_array,
        y=sc_input,
        mode='lines',
        name='sc_input (time)'
    ),
    row=1, col=1
)

# 3) Bottom subplot: Spectrogram (Heatmap)
fig_input.add_trace(
    go.Heatmap(
        z=spec_in_db,
        x=times_input,
        y=frequencies_input,
        colorscale='Viridis',
        zmin=z_scale_min_max[0],
        zmax=z_scale_min_max[1],
        #coloraxis='coloraxis',  # tie color scale to a single color axis
        name='Spectrogram'
    ),
    row=2, col=1
)

# 4) Update layout, axis labels, colorbars
fig_input.update_layout(
    title='sc_input: Time-Domain & Spectrogram (Linked)',
    coloraxis=dict(
        colorbar=dict(title='Intensity [dB]')
    ),
    xaxis=dict(title='Time [s]'),     # Top subplot x-axis
    xaxis2=dict(title='Time [s]'),    # Bottom subplot x-axis
    yaxis=dict(title='Amplitude'),    # Top subplot y-axis
    yaxis2=dict(title='Frequency [Hz]'),  # Bottom subplot y-axis
    height=800
    
)

fig_input.show()


In [ ]:
if sc_cut_in_sec is not None:
    sc_cut_input, sc_cut_output = sct.cut_signals(sc_input, sc_output, sample_rate*sc_cut_in_sec)
else:
    # cut to the minimum length
    min_length = min(len(sc_input), len(sc_output))
    sc_cut_output = sc_output[:min_length]
    sc_cut_input = sc_input[:min_length]
    print(f"cut sc_cut_length: {min_length}")

shift = sct.find_shift_between_signals(sc_cut_output[shift_between_signals_range], 
                             sc_cut_input[shift_between_signals_range],  
                             sample_rate=sample_rate,
                            interpolation_factor=interpolated_shift_factor ,
                            corr_mode='full',
                            method='correlate',
                            plot=False)
print(f'Estimated shift: {shift:.3f} samples')

# Compare the final aligned signal2 with sig1 if desired
# (They should be nearly identical apart from edge effects.)
method = 'cubic'

sc_cut_input_shifted = sct.fractional_time_shift(sc_cut_input, shift, method=method)
# Plot all the sc_cut variables for comparison
show_plot = False
if show_plot:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=sc_cut_output, mode='lines', name='sc_output'))
    fig.add_trace(go.Scatter(y=sc_cut_input, mode='lines', name='sc_input'))
    fig.add_trace(go.Scatter(y=sc_cut_input_shifted, mode='lines', name=f"sc_input shifted by {shift:.1f} samples ({method})"))
    fig.update_layout(title='Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
    fig.show()





In [ ]:
import numpy as np
from scipy.signal import hilbert
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Compute the time array
t = np.arange(len(sc_cut_output)) / sample_rate

# Compute the analytic signals via Hilbert transform
analytic_signal_1 = hilbert(sc_cut_output)
analytic_signal_2 = hilbert(sc_cut_input_shifted)

# Compute the instantaneous phases
phase_1 = np.unwrap(np.angle(analytic_signal_1))
phase_2 = np.unwrap(np.angle(analytic_signal_2))

# find the phase difference between the samples of signal 1
phase_diff_1 = np.diff(phase_1)
phase_diff_2 = np.diff(phase_2)

# Compute the phase difference
phase_diff_2_1 = phase_2 - phase_1

# Suppose phase_diff_1 and phase_diff_2 are the per-sample phase differences (in radians)
# and sample_rate is in samples/second



# Convert radians/sample to Hz
freq_1 = (phase_diff_1 / (2.0 * np.pi)) * sample_rate
freq_2 = (phase_diff_2 / (2.0 * np.pi)) * sample_rate

# freq_1 and freq_2 now represent the instantaneous frequency (in Hz)

# Create subplots: top row for phase difference, bottom row for signals
fig_input = make_subplots(
    rows=2, 
    cols=1,
    shared_xaxes=True,   # Link the x-axis for zooming
    vertical_spacing=0.12,
    specs=[
        [{"secondary_y": True}],  # Top subplot: dual y-axis
        [{}]                      # Bottom subplot: single y-axis
    ]
)


# 1) Add the phase difference trace in the top subplot
fig_input.add_trace(
    go.Scatter(
        x=t[plot_range],
        y=phase_diff_2_1[plot_range],
        mode='lines',
        name='Phase Difference between Sig 2 and Sig 1',
    ),
    row=1, col=1, secondary_y=True
)


# add the phase difference between the samples of signal 1
fig_input.add_trace(
    go.Scatter(
#        x=t[:-1],
        #y=freq_1,
        x=t[plot_range[:-1]],
        y=freq_1[plot_range[:-1]],        
        mode='lines',
        name='Sig 1 Instanenous Freq'
    ),
    row=1, col=1, secondary_y=False
)

fig_input.add_trace(
    go.Scatter(
#        x=t[:-1],
        x=t[plot_range[:-1]],
        y=freq_2[plot_range[:-1]],
#        y=freq_2,
        mode='lines',
        name='Sig 2 Instanenous Freq'
    ),
    row=1, col=1, secondary_y=False
)

# 2) Add both signals in the bottom subplot
fig_input.add_trace(
    go.Scatter(
        x=t[plot_range],
        y=sc_cut_output[plot_range],
        mode='lines',
        name='Signal 1: Input/Output',
        line=dict(width=4)  # Adjust the width as desired
    ),
    row=2, col=1
)

fig_input.add_trace(
    go.Scatter(
        x=t[plot_range],
        y=sc_cut_input_shifted[plot_range],
        mode='lines',
        name='Signal 2: Shifted'
    ),
    row=2, col=1
)

# Update layout: titles and labels
fig_input.update_layout(
    title=f'Phase Difference and Signal Comparison for {filename}',
    xaxis_title='Time (s)',
    yaxis_title='Frequency (Hz)',
    # The second subplot's y-axis label can be set via yaxis2_title or by a separate annotation.
    yaxis2_title='Phase Difference (rad)'
)

# Update axis limits
# Custom limits for phase_diff_2_1
r1 = min(np.min(freq_1[plot_range[:-1]]),np.min(freq_2[plot_range[:-1]]))
r2 = max(np.max(freq_1[plot_range[:-1]]),np.max(freq_2[plot_range[:-1]]))

freq_diff_bounds_hz = [r1, r2]

if freq_diff_bounds_hz is not None:
    fig_input.update_yaxes(range=freq_diff_bounds_hz, row=1, col=1, secondary_y=False)  # Frequency range in Hz

# find the min and max bounds of the phase_diff_2_1 values after the first 100 samples
phase_diff_bounds_rad = [np.min(phase_diff_2_1[plot_range[:-1]]), np.max(phase_diff_2_1[plot_range[:-1]])]

fig_input.update_yaxes(range=phase_diff_bounds_rad, row=1, col=1, secondary_y=True)  # Phase difference range in radians


# Show the figure
fig_input.show()

